In [51]:
import sys
moduleLoc = "/Users/Lykke-AndersenLab/PycharmProjects/AnoThER-Seq"
if moduleLoc in sys.path: 
    print('woo')
else:
    sys.path.append(moduleLoc)
    import matplotlib.pyplot as plt
    
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

# Enable inline plotting
%matplotlib inline

woo


In [56]:
#read in tail files
folderLoc = "/Users/Lykke-AndersenLab/Dropbox/NoctData/"
siLuc = pd.read_csv(folderLoc+"siLuc.tails")
siNoct = pd.read_csv(folderLoc+"siNoct.tails")
NoctWT = pd.read_csv(folderLoc+"NoctWT.tails")
NoctEA = pd.read_csv(folderLoc+"NoctEA.tails")
print(siLuc.columns)
print(siNoct.columns)
print(NoctWT.columns)
print(NoctEA.columns)

Index(['Sequence,UniqueReads,Gene,3Loc,TailLength,TailSequence'], dtype='object')
Index(['Sequence,UniqueReads,Gene,3Loc,TailLength,TailSequence'], dtype='object')
Index(['Sequence,UniqueReads,Gene,3Loc,TailLength,TailSequence'], dtype='object')
Index(['Sequence,UniqueReads,Gene,3Loc,TailLength,TailSequence'], dtype='object')


In [60]:
siLuc

,"Sequence,UniqueReads,Gene,3Loc,TailLength,TailSequence"
0,CAGGGCGAGGCTTATCCATTGCACTCCGGATGTGCTGACCCCTGCG...
1,ATCCGAGGACAATATATTAAATGGATTTTTGGAGCAGGGAGATGGA...
2,GTGCTCGCTTCGGCAGCACATATACTAAAATTGGAACGATACAGAG...
3,CTATCCGAGGACAATATATTAAATGGATTTTTGGAGCAGGGAGATG...
4,CAATGAGGTTTATCCGAGGCGCGATTATTGCTAATTGAAAACTTTT...
5,ATACTCTGGTTTCTCTTCAGATCGCATAAATCTTTCGCCTTTTACT...
6,CCTGAGCGTGAAGCCGGCTTTCTGGCGTTGCTTGGCTGCAACTGCC...
7,ATACTCTGGTTTCTCTTCAGATCGTATAAATCTTTCGCCTTTTACT...
8,ATCCGAGGACAATATATTAAATGGATTTTTGGAGCAGGGAGATGGA...
9,GTGCTCGCTTCGGCAGCACATATACTAAAATTGGAACGATACAGAG...
